# Lesson 2 : LangGraph Components

In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


> If you are not familiar with python typing annotation, you can refer to the [python documents](https://docs.python.org/3/library/typing.html).

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

> Note: in `take_action` below, some logic was added to cover the case that the LLM returned a non-existent tool name. Even with function calling, LLMs can still occasionally hallucinate. Note that all that is done is instructing the LLM to try again! An advantage of an agentic organization.

In [5]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-4o-mini")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [10]:
# from IPython.display import Image

# Image(abot.graph.get_graph().draw_png())

In [11]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather San Francisco'}, 'id': 'call_PsSmuuHUUyuHMFvSsm4qJf0S', 'type': 'tool_call'}
Back to the model!


In [12]:
result

{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PsSmuuHUUyuHMFvSsm4qJf0S', 'function': {'arguments': '{"query":"current weather San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 151, 'total_tokens': 172}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ee120421-1bbb-418d-ab67-47d2814db485-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather San Francisco'}, 'id': 'call_PsSmuuHUUyuHMFvSsm4qJf0S', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 21, 'total_tokens': 172}),
  ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'co

In [13]:
result['messages'][-1].content

'The current weather in San Francisco is as follows:\n\n- **Temperature**: 66°F (approximately 19°C)\n- **Condition**: Partly cloudy\n- **Humidity**: 73%\n- **Wind**: 4.3 mph from the SSW\n- **Visibility**: 16 km\n- **Pressure**: 1015 mb\n\nFor more details, you can check the full weather report [here](https://www.weatherapi.com/).'

In [14]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today'}, 'id': 'call_X6MBkVVcJlTNb3Wf9WosXIyB', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Los Angeles weather today'}, 'id': 'call_zqpU1amFAkg25AIuzB1LhvSm', 'type': 'tool_call'}
Back to the model!


In [16]:
print(result['messages'][-1].content)

### Current Weather:

#### San Francisco, CA
- **Temperature**: 18.9°C (66.0°F)
- **Condition**: Partly cloudy
- **Humidity**: 73%
- **Wind**: 4.3 mph (6.8 kph) from the SSW
- **Pressure**: 1015 mb
- **Visibility**: 16 km (9 miles)

#### Los Angeles, CA
- **Temperature**: 24.4°C (75.9°F)
- **Condition**: Sunny
- **Humidity**: 69%
- **Wind**: 3.8 mph (6.1 kph) from the NW
- **Pressure**: 1013 mb
- **Visibility**: 13 km (8 miles)

For more details, you can check the weather forecast for [San Francisco](https://www.weatherapi.com/) and [Los Angeles](https://www.weatherapi.com/).


In [17]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o-mini")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': '2024 Super Bowl winner'}, 'id': 'call_lit4wWA2pATAOqnSJxHqkjVa', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Kansas City Chiefs headquarters location'}, 'id': 'call_FgYZ4hno5t9fGs4FKKysZ1Wz', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Missouri 2024'}, 'id': 'call_ll4A9JjZzPvHFsBdPKfATclL', 'type': 'tool_call'}
Back to the model!


In [18]:
print(result['messages'][-1].content)

1. **Super Bowl Winner 2024**: The Kansas City Chiefs won the Super Bowl in 2024, defeating the San Francisco 49ers with a score of 25-22 in overtime.

2. **Location of Winning Team Headquarters**: The Kansas City Chiefs are headquartered in Kansas City, Missouri.

3. **GDP of Missouri**: As of 2023, the GDP of Missouri was approximately $344.12 billion. 

If you have any more questions or need further information, feel free to ask!
